# Homework set 3

Please **submit this Jupyter notebook through Canvas** no later than **Monday December 2**. **Submit the notebook file with your answers (as .ipynb file) and a pdf printout. The pdf version can be used by the teachers to provide feedback. On canvas there are hints about creating a nice pdf version.**

Before you hand in, please make sure the notebook runs, by running "Restart kernel and run all cells..." from the Kernel menu.

Homework is in **groups of two**, and you are expected to hand in original work. Work that is copied from another group will not be accepted.

# Exercise 0
Costanza D'Ercole - 15424596

Zoe Azra Blei - 15762467

Run the following cell to import NumPy, Matplotlib. If anything else is needed you can import this yourself.

In [1]:
import numpy as np
import matplotlib.pyplot as plt


# Exercise 1: Nonlinear least squares

This exercise is about the Gauss-Newton method, and the Levenberg-Marquardt method, which are discussed in section 6.6 of Heath. Please read this section before making this homework set. **In this exercise set the Levenberg-Marquardt method is a little different from the one in Heath. The first equation in subsection 6.6.2 is replaced by**
$$
\renewcommand{\bfA}{\boldsymbol{A}}
\renewcommand{\bfB}{\boldsymbol{B}}
\renewcommand{\bfJ}{\boldsymbol{J}}
\renewcommand{\bfr}{\boldsymbol{r}}
\renewcommand{\bfs}{\boldsymbol{s}}
\renewcommand{\bfx}{\boldsymbol{x}}
\renewcommand{\for}{\text{\bf for }}
\renewcommand{\end}{\text{\bf end }}
\bigg(\bfJ^T(\bfx_k) \bfJ(\bfx_k) 
+ \mu_k \operatorname{Diagonal}\big( \bfJ^T(\bfx_k) \bfJ(\bfx_k) \big) \bigg) \bfs_k
= - \bfJ^T(\bfx_k) \, \bfr(\bfx_k)
$$
Here $\operatorname{Diagonal}(\bfB)$ denotes the diagonal part of $\bfB$. So $\operatorname{Diagonal}(\bfB)$ has the same shape as $\bfB$ and identical entries on the diagonal and it has zero off-diagonal entries.

The algorithm for Levenberg-Marquardt, with $\mu_k$ constant (denoted $\mu$ here), is then</br>
$\qquad \bfx_0 = \text{initial guess}$</br>
$\qquad \mu = \text{constant}$</br>
$\qquad \for k = 0,1,2, \ldots$</br>
$\qquad \qquad \bfA = \bfJ_f(\bfx_k)$</br>
$\qquad \qquad \text{solve } \bfs_k \text{ from } 
(\bfA^T \bfA + \mu \operatorname{Diagonal}(\bfA^T \bfA)) \bfs_k = - \bfA^T \bfr(\bfx_k)$</br>
$\qquad \qquad \bfx_{k+1} = \bfx_k + \bfs_k$</br>
$\qquad \end$</br>

This reduces to the Gauss-Newton method if $\mu = 0$. 

## (a)
Implement the Levenberg-Marquardt method with constant $\mu$ using a suitable stopping criterion. 
Make it such that the user can specify the value of the tolerance in the stopping criterion via a parameter `tol` and the maximum number of iterations via a
parameter `maxIter`. In the implementation you can use library functions for linear algebra operations. 

In [81]:
def Lev_Mar(x0, mu, tol, maxIter, f, J, b, t):
    """
    Levenberg-Marquardt method for nonlinear least squares.

    Parameters:
        x0: Initial guess (array-like).
        mu: Damping parameter (scalar).
        tol: Tolerance for stopping criterion.
        maxIter: Maximum number of iterations.
        f: Function for computing f(x).
        J: Function for computing Jacobian J_f(x).
        b: Observed data vector.

    Returns:
        x_new: Solution vector.
    """
    x_old = x0
    iters = 0
    for _ in range(maxIter):
        A = J(x_old, t)  # Compute Jacobian matrix J_f(x_k)
        B = np.diag(np.diag(A.T @ A))  # Compute Diagonal(A^T A)
        r = f(x_old, t) - b  # Compute residual r(x_k)
        s = np.linalg.solve(A.T @ A + mu * B, -A.T @ r)  # Solve for s_k
        
        x_new = x_old + s  # Update x_k+1
        
        # Check stopping criterion
        if np.linalg.norm(x_new - x_old) < tol:
            return x_new, iters + 1
        
        x_old = x_new
        iters += 1
    
    # Raise exception if not converged within maxIter
    raise Exception("Did not converge in max iterations!")


## (b) 

The time course of drug concentration $y$ in the bloodstream is well described by
$$ \tag{1}
  y = c_1 t e^{c_2 t} ,
$$
where $t$ denotes time after the drug was administered. The characteristics of the model
are a quick rise as the drug enters the bloodstream, followed by slow exponential decay.
The half-life of the drug is the time from the peak concentration to the time it drops to
half that level. The measured level of the drug norfluoxetine in a patient's bloodstream at whole hours after it was administered is given in the following data:

In [82]:
# time in hours
hour = np.array( [ 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0 ] )
# concentration in ng/ml
concentration = np.array( [ 8.0, 12.3, 15.5, 16.8, 17.1, 15.8, 15.2, 14.0 ] )


Use the Gauss-Newton method to fit this data to the blood concentration model (1).

Also use the Levenberg-Marquardt method with $\mu =0.1$ to address the same problem.

Which method produces the least number of iterations? N.B. clearly state the starting point. 

You are asked to use your own version of Gauss-Newton and Levenberg-Marquardt.

In [83]:
# Gauss-Newton function
def Gauss_Newton(x0, tol, maxIter, f, J, b, t):
    x_old = x0
    iters = 0
    for _ in range(maxIter):
        A = J(x_old, t)  # Compute Jacobian matrix J_f(x_k)
        r = f(x_old, t) - b  # Compute residual r(x_k)
        s = np.linalg.solve(A.T @ A, -A.T @ r)  # Solve for s_k
        
        x_new = x_old + s  # Update x_k+1
        
        # Check stopping criterion
        if np.linalg.norm(x_new - x_old) < tol:
            return x_new, iters + 1
        
        x_old = x_new
        iters += 1
    
    # Raise exception if not converged within maxIter
    raise Exception("Did not converge in max iterations!")

x0 = [-1,1]
mu = 0.1
tol = 1e-6
maxIter = 50

def f(x, t):
    x1, x2 = x
    return x1 * t * np.exp(x2 * t)

def J(x, t):
    x1, x2 = x
    return np.column_stack((
        t * np.exp(x2 * t),          # Partial derivative w.r.t x1
        x1 * t**2 * np.exp(x2 * t)  # Partial derivative w.r.t x2
    ))

print(f"The starting values for c1 and c2 are: {x0}")
# Fit with Levenberg-Marquart method
sol, iters = Lev_Mar(x0, mu, tol, maxIter, f, J, concentration, hour)
print(f"Approximate solution with Levenberg-Marquard: {sol}")
print(f"Solution reached in {iters} iterations.")

# Fiit with Gauss-Newton
sol_g, iters_g = Gauss_Newton(x0, tol, maxIter, f, J, concentration, hour)
print(f"Approximate solution with Gauss-Newton: {sol_g}")
print(f"Solution reached in {iters_g} iterations.")



The starting values for c1 and c2 are: [-1, 1]
Approximate solution with Levenberg-Marquard: [ 9.79692699 -0.21508714]
Solution reached in 46 iterations.
Approximate solution with Gauss-Newton: [ 9.79692816 -0.21508717]
Solution reached in 17 iterations.


Looking at the results, the Gauss-Newton method converged faster, with less iterations required (17) compared to Levenberg-Marquard (46).

# (c)
Try to find a starting point such that Gauss-Newton does not converge, while Levenberg-Marquardt does.



In [84]:
x0 = [2 ,1]

print(f"The starting values for c1 and c2 are: {x0}")
# Fit with Levenberg-Marquart method
sol, iters = Lev_Mar(x0, mu, tol, maxIter, f, J, concentration, hour)
print(f"Approximate solution with Levenberg-Marquard: {sol}")
print(f"Solution reached in {iters} iterations.")

# Fiit with Gauss-Newton
sol_g, iters_g = Gauss_Newton(x0, tol, maxIter, f, J, concentration, hour)
print(f"Approximate solution with Gauss-Newton: {sol_g}")
print(f"Solution reached in {iters_g} iterations.")


The starting values for c1 and c2 are: [2, 1]
Approximate solution with Levenberg-Marquard: [ 9.79692732 -0.21508715]
Solution reached in 47 iterations.


Exception: Did not converge in max iterations!

With starting values [2, 1], the Levenberg-Marquard method converged to an approximate solution in 47 iterations. 

However, the Gauss-Newton method did not converge in the maximum number of iterations allowed.

This is most likely due to the non-linear function f: given the presence of an exponential the function may produce large residuals for some values of
c1 and c2, leading to a potentially ill-conditioned Jacobian matrix. This may cause the method to diverge.

In contrast, the parameter $\mu$ in the Levenberg-Marquard model stabilizes the updates, allowing the method to converge even when the starting guess is far from the true solution,
or when the residuals are large.

# (d) 
So far for simplicity, we considered constant $\mu$. However
Levenberg-Marquardt is often applied adaptively with a varying $\mu$. 
A common strategy is to continue to decrease $\mu$ by a factor of 10 on each iteration step as long as the residual sum of squared errors is decreased by the step, and if the sum increases, to reject the step and increase $\mu$ by a factor of 10.

Implement an adaptive variant of Levenberg-Marquardt using such a strategy for choosing $\mu$. 

Compare the performance (iteration number) of the adaptive variant with Gauss-Newton and the previous, non-adaptive variant of Levenberg-Marquardt. Consider a starting point for which Gauss-Newton converged rapidly and a starting point for which Gauss-Newton did not converge, but non-adaptive Levenberg-Marquardt did. Give your answer in a table for clarity, also indicating the starting point and if relevant other parameters.

In [156]:
def Lev_Mar_modified(x0, mu, tol, maxIter, f, J, b, t):
    """
    Adaptive Levenberg-Marquardt method for nonlinear least squares.

    Parameters:
        x0: Initial guess (array-like).
        mu: Initial damping parameter (scalar).
        tol: Tolerance for stopping criterion.
        maxIter: Maximum number of iterations.
        f: Function for computing f(x).
        J: Function for computing Jacobian J_f(x).
        b: Observed data vector.
        t: Independent variable(s).

    Returns:
        x_new: Solution vector.
        iters: Number of iterations.
    """
    x_old = x0
    iters = 0
    rss_old = np.inf  # Initialize RSS (residual sum of squares)
    
    for i in range(maxIter):
        A = J(x_old, t)  # Compute Jacobian matrix J_f(x_k)
        B = np.diag(np.diag(A.T @ A))  # Compute Diagonal(A^T A)
        r = f(x_old, t) - b  # Compute residual vector r(x_k)
        rss_new = np.linalg.norm(r)**2  # Compute RSS for the current step
        
        # If RSS increases, increase mu and do not update x
        if rss_new > rss_old:
            mu *= 10
            continue  # Reject the step
        
        # Otherwise, decrease mu and accept the step
        mu /= 10
        s = np.linalg.solve(A.T @ A + mu * B, -A.T @ r)  # Solve for s_k
        
        x_new = x_old + s  # Update x_k+1
        
        # Check stopping criterion
        if np.linalg.norm(x_new - x_old) < tol:
            return x_new, iters + 1
        
        # Update for next iteration
        x_old = x_new
        rss_old = rss_new  # Update the previous RSS
        iters += 1
    
    # Raise exception if not converged within maxIter
    raise Exception("Did not converge in max iterations!")

mu = 0.1
tol = 1e-6
maxIter = 50
x0 = (-10, 0)

print(f"The starting values for c1 and c2 are: {x0}")

# Fit with Levenberg-Marquart method
sol_l1, iters_l1 = Lev_Mar(x0, mu, tol, maxIter, f, J, concentration, hour)
print(f"Approximate solution with Levenberg-Marquard: {sol_l1}")
print(f"Solution reached in {iters_l1} iterations.")

# Fit with Levenberg-Marquart modified method
sol_lm1, iters_lm1 = Lev_Mar_modified(x0, mu, tol, maxIter, f, J, concentration, hour)
print(f"Approximate solution with Levenberg-Marquard adaptive: {sol_lm1}")
print(f"Solution reached in {iters_lm1} iterations.")

# Fit with Gauss-Newton
sol_g1, iters_g1 = Gauss_Newton(x0, tol, maxIter, f, J, concentration, hour)
print(f"Approximate solution with Gauss-Newton: {sol_g1}")
print(f"Solution reached in {iters_g1} iterations.")



The starting values for c1 and c2 are: (-10, 0)
Approximate solution with Levenberg-Marquard: [ 9.79692687 -0.21508714]
Solution reached in 31 iterations.
Approximate solution with Levenberg-Marquard adaptive: [ 9.79692815 -0.21508717]
Solution reached in 7 iterations.
Approximate solution with Gauss-Newton: [ 9.79692815 -0.21508717]
Solution reached in 8 iterations.


Taking (1,0) as starting values, only Levenberg-Marquard method converged, while Gauss-Newton was stopped due to the presence of a singular Hessian matrix.

|Method| Starting Values | Tolerance (tol) | Mu | Max Iterations | Convergence | Iterations for convergence |
|-----|------------|-----------------|----|----------------|-------------|--------|
|Gauss-Newton| [1, 0]          | 1e-6           | 0.1| 50             | No         |-|
|Levenberg-Marquard| [1, 0]          | 1e-6           | 0.1| 50             | Yes          |31|
|Adaptive Levenberg-Marquard| [1, 0]          | 1e-6           |Adaptive| 50             | No         |-|


However, with (-10, 0) as starting value, all three methods converged. Gauss-Newton and the adaptive Levenberg-Marquard conevrged fastly with 8 and 7 iterations, respectively. 
Levenberg-Marquard converged slowly, requiring 31 iterations.

|Method| Starting Values | Tolerance (tol) | Mu | Max Iterations | Convergence | Iterations for convergence |
|-----|------------|-----------------|----|----------------|-------------|--------|
|Gauss-Newton| [-10, 0]          | 1e-6           | 0.1| 50             | Yes         |8|
|Levenberg-Marquard| [-10, 0]          | 1e-6           | 0.1| 50             | Yes          |31|
|Adaptive Levenberg-Marquard| [-10, 0]          | 1e-6           |Adaptive| 50             | Yes         |7|

